In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.losses import CategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import StandardScaler

from keras.layers import Input, Dense
from keras.models import Model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

In [14]:
tf.config.optimizer.set_jit(True)

np.random.seed(0)

In [15]:
data = pd.read_csv('./SDSS/star_classification.csv')
cols = ['alpha','delta','u','g','r','i','z','redshift','class']
data = data[cols]
data.head()

,alpha,delta,u,g,r,i,z,redshift,class
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,GALAXY
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,GALAXY
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,GALAXY
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,GALAXY


Se cambian las clases a números enteros

$$ \text{GALAXY}: 0,\quad \text{STAR}: 1,\quad \text{QSO}: 2$$

In [16]:
data["class"]=[0 if i == "GALAXY" else 1 if i == "STAR" else 2 for i in data["class"]]
print(data.head())
data = data.to_numpy()

        alpha      delta         u         g         r         i         z  \
0  135.689107  32.494632  23.87882  22.27530  20.39501  19.16573  18.79371   
1  144.826101  31.274185  24.77759  22.83188  22.58444  21.16812  21.61427   
2  142.188790  35.582444  25.26307  22.66389  20.60976  19.34857  18.94827   
3  338.741038  -0.402828  22.13682  23.77656  21.61162  20.50454  19.25010   
4  345.282593  21.183866  19.43718  17.58028  16.49747  15.97711  15.54461   

   redshift  class  
0  0.634794      0  
1  0.779136      0  
2  0.644195      0  
3  0.932346      0  
4  0.116123      0  


### Implementation

We will use wind power forecast data, which is available at the following link. It consists of normalized (between zero and one) wind power measurements from seven wind farms. To keep things simple, we will use first wind farm data (column named wp1) but I encourage the reader to experiment and extend the code to forecast energy for all seven, wind farms.

Let’s import required packages, load the dataset and define two helper functions. The first method prepare_dataset will segment the data into chunks to create X, Y pair for model training. The X will the wind power values from the past (e.g. 1 to t-1) and Y will be future value at time t. The second method train_evaluate perform three things, 1) decoding GA solution to get window size and number of units. 2) Prepare the dataset using window size found by GA and divide into train and validation set, and 3) train LSTM model, calculate RMSE on validation set and return it as a fitness score of the current GA solution.

In [17]:
data[:,1].shape
data[:,0:3].shape[1]

3

In [18]:
2**len(np.arange(15)[0:2])

4

In [ ]:
SC_LAYERS = hp.HParam('layers', hp.Discrete([50, 100, 150]))
SC_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150]))
# SC_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.0))
SC_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8]))

In [70]:
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:,0:8]
    Y = data[:,8]
    Y = to_categorical(Y, num_classes=3)
    return X, Y

def train_evaluate(ga_individual_solution):   
    t = time.time()
    
    # Decode GA solution to integer for window_size and num_units
    deep_size_bits = BitArray(ga_individual_solution[0:2]) # 4
    num_units_bits = BitArray(ga_individual_solution[2:4]) # 4
    learning_rate_bits = BitArray(ga_individual_solution[4:7]) # 8
    batch_size_bits = BitArray(ga_individual_solution[7:8]) # 4
    
    deep_size = 4*(deep_size_bits.uint + 1)
    num_units = 4*(num_units_bits.uint + 1)
#     num_units = 50*(num_units_bits.uint + 1)
    learning_rate = 8*(learning_rate_bits.uint+1)*10**(-4)
#     learning_rate = 10*(learning_rate_bits.uint+1)*10**(-4)
    batch_size = 2**(batch_size_bits.uint + 4)
#     batch_size = 2**(batch_size_bits.uint +1)
    epochs = 10
    
    
    print('\nDeep Size: ', deep_size,', Num of Units: ',num_units,', Learning rate: ',learning_rate,', Batch Size: ',batch_size)
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    X,Y = prepare_dataset(data)
    X_train, X_test, Y_train, Y_test = split(X, Y, test_size = 0.3, random_state = 0)
    X_train = scalar.fit_transform(X_train)
    X_test  = scalar.transform(X_test)
    
    # Train LSTM model and predict on validation set
    model = keras.Sequential()
    model.add(keras.Input(shape=(int(X_train.shape[1]),)))
    model.add(layers.Dense(num_units, input_shape=(int(X_train.shape[1]),)))
    
    for i in range(deep_size):        
        model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dense(3, activation=tf.nn.softmax))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=["categorical_accuracy"])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',
                                   min_delta=0,
                                   patience=5,
                                   restore_best_weights=True)]
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
              epochs=epochs, callbacks=callbacks, batch_size=batch_size, shuffle=True)
    Y_pred = model.predict(X)
    Y_pred = np.argmax(Y_pred, axis=1)
    Y = np.argmax(Y, axis=1)
    t = time.time()-t
    
#     Calculate the RMSE score as fitness score for GA
#     while True:
#         try:
    error = accuracy_score(Y, Y_pred)
    print('Validation CEE: ', error)
    print('Time passed: ', t)
#         except ValueError:
#             error = 10000
#             break
    datos.append([deep_size, num_units, learning_rate, batch_size, epochs, error, t])
    
    return error,

A continuación, use la paquetería DEAP para definir las cosas para ejecutar GA. Usaremos una representación binaria para la solución de longitud diez. Se inicializará aleatoriamente utilizando la distribución de Bernoulli. Del mismo modo, se utiliza el crossover ordenado, la mutación aleatoria y la selección de la rueda de la ruleta. Los valores del parámetro GA se inicializan arbitrariamente; Te sugerimos que juegues con diferentes configuraciones.



In [71]:
population_size = 10
num_generations = 10
gene_length = 8

datos = []

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = [1.0])
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, 
n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
                        ngen = num_generations, verbose = True)

C:\Users\Joshu\.conda\envs\tensorflow\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joshu\.conda\envs\tensorflow\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)



Deep Size:  12 , Num of Units:  8 , Learning rate:  0.0032 , Batch Size:  16
Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 42s 594us/sample - loss: 0.2647 - categorical_accuracy: 0.9070 - val_loss: 0.3526 - val_categorical_accuracy: 0.9466
Epoch 2/10
70000/70000 [==============================] - 35s 504us/sample - loss: 0.1557 - categorical_accuracy: 0.9509 - val_loss: 1.1646 - val_categorical_accuracy: 0.9592
Epoch 3/10
70000/70000 [==============================] - 43s 618us/sample - loss: 0.1435 - categorical_accuracy: 0.9559 - val_loss: 0.7343 - val_categorical_accuracy: 0.9645
Epoch 4/10
70000/70000 [==============================] - 34s 490us/sample - loss: 0.1402 - categorical_accuracy: 0.9573 - val_loss: 1.2913 - val_categorical_accuracy: 0.9619
Epoch 5/10
70000/70000 [==============================] - 33s 464us/sample - loss: 0.1366 - categorical_accuracy: 0.9585 - val_loss: 0.5910 - val_categorical_accuracy: 0.96

70000/70000 [==============================] - 18s 261us/sample - loss: 0.2121 - categorical_accuracy: 0.9371 - val_loss: 0.2389 - val_categorical_accuracy: 0.9174
Epoch 9/10
70000/70000 [==============================] - 22s 317us/sample - loss: 0.3046 - categorical_accuracy: 0.8573 - val_loss: 0.1725 - val_categorical_accuracy: 0.9569
Validation CEE:  0.27346
Time passed:  182.92317724227905

Deep Size:  4 , Num of Units:  16 , Learning rate:  0.0064 , Batch Size:  16
Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 37s 523us/sample - loss: 0.1983 - categorical_accuracy: 0.9307 - val_loss: 0.2436 - val_categorical_accuracy: 0.9532
Epoch 2/10
70000/70000 [==============================] - 34s 482us/sample - loss: 0.1534 - categorical_accuracy: 0.9507 - val_loss: 0.8325 - val_categorical_accuracy: 0.9443
Epoch 3/10
70000/70000 [==============================] - 33s 468us/sample - loss: 0.1443 - categorical_accuracy: 0.9541 - va

70000/70000 [==============================] - 23s 331us/sample - loss: 0.1314 - categorical_accuracy: 0.9590 - val_loss: 2.0934 - val_categorical_accuracy: 0.9586
Epoch 5/10
70000/70000 [==============================] - 24s 341us/sample - loss: 0.1295 - categorical_accuracy: 0.9595 - val_loss: 2.2957 - val_categorical_accuracy: 0.9598
Epoch 6/10
70000/70000 [==============================] - 27s 388us/sample - loss: 0.1273 - categorical_accuracy: 0.9602 - val_loss: 1.1416 - val_categorical_accuracy: 0.9647
Epoch 7/10
70000/70000 [==============================] - 30s 426us/sample - loss: 0.1269 - categorical_accuracy: 0.9600 - val_loss: 1.7922 - val_categorical_accuracy: 0.9570
Epoch 8/10
70000/70000 [==============================] - 30s 427us/sample - loss: 0.1233 - categorical_accuracy: 0.9610 - val_loss: 5.6322 - val_categorical_accuracy: 0.9669
Epoch 9/10
70000/70000 [==============================] - 30s 424us/sample - loss: 0.1252 - categorical_accuracy: 0.9616 - val_loss: 2.1

70000/70000 [==============================] - 42s 594us/sample - loss: 0.1576 - categorical_accuracy: 0.9515 - val_loss: 0.4424 - val_categorical_accuracy: 0.9639
Epoch 8/10
70000/70000 [==============================] - 42s 593us/sample - loss: 0.1596 - categorical_accuracy: 0.9512 - val_loss: 1.4153 - val_categorical_accuracy: 0.9634
Epoch 9/10
70000/70000 [==============================] - 41s 589us/sample - loss: 0.1338 - categorical_accuracy: 0.9604 - val_loss: 0.2545 - val_categorical_accuracy: 0.9647
Epoch 10/10
70000/70000 [==============================] - 41s 584us/sample - loss: 0.1305 - categorical_accuracy: 0.9603 - val_loss: 0.3377 - val_categorical_accuracy: 0.9631
Validation CEE:  0.38806
Time passed:  430.95746970176697

Deep Size:  12 , Num of Units:  4 , Learning rate:  0.0008 , Batch Size:  16
Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 42s 594us/sample - loss: 0.4796 - categorical_accuracy: 0.8331 - v

...

70000/70000 [==============================] - 15s 219us/sample - loss: 0.1519 - categorical_accuracy: 0.9569 - val_loss: 0.1567 - val_categorical_accuracy: 0.9543
Epoch 5/10
70000/70000 [==============================] - 15s 219us/sample - loss: 0.1439 - categorical_accuracy: 0.9588 - val_loss: 0.1343 - val_categorical_accuracy: 0.9627
Epoch 6/10
70000/70000 [==============================] - 15s 220us/sample - loss: 0.1398 - categorical_accuracy: 0.9604 - val_loss: 0.1305 - val_categorical_accuracy: 0.9633
Epoch 7/10
70000/70000 [==============================] - 15s 217us/sample - loss: 0.1350 - categorical_accuracy: 0.9618 - val_loss: 0.1268 - val_categorical_accuracy: 0.9634
Epoch 8/10
70000/70000 [==============================] - 15s 219us/sample - loss: 0.1339 - categorical_accuracy: 0.9625 - val_loss: 0.1335 - val_categorical_accuracy: 0.9592
Epoch 9/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1320 - categorical_accuracy: 0.9629 - val_loss: 0.1

70000/70000 [==============================] - 15s 217us/sample - loss: 0.1608 - categorical_accuracy: 0.9488 - val_loss: 0.5521 - val_categorical_accuracy: 0.9263
Epoch 3/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1476 - categorical_accuracy: 0.9543 - val_loss: 0.4568 - val_categorical_accuracy: 0.9486
Epoch 4/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1451 - categorical_accuracy: 0.9551 - val_loss: 0.2815 - val_categorical_accuracy: 0.9569
Epoch 5/10
70000/70000 [==============================] - 15s 217us/sample - loss: 0.1401 - categorical_accuracy: 0.9570 - val_loss: 0.2560 - val_categorical_accuracy: 0.9588
Epoch 6/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1364 - categorical_accuracy: 0.9585 - val_loss: 0.1549 - val_categorical_accuracy: 0.9579
Epoch 7/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1373 - categorical_accuracy: 0.9577 - val_loss: 0.1

70000/70000 [==============================] - 33s 477us/sample - loss: 0.1516 - categorical_accuracy: 0.9542 - val_loss: 0.1266 - val_categorical_accuracy: 0.9626
Epoch 5/10
70000/70000 [==============================] - 32s 461us/sample - loss: 0.1498 - categorical_accuracy: 0.9539 - val_loss: 0.1503 - val_categorical_accuracy: 0.9501
Epoch 6/10
70000/70000 [==============================] - 34s 488us/sample - loss: 0.1448 - categorical_accuracy: 0.9551 - val_loss: 0.1298 - val_categorical_accuracy: 0.9602
Epoch 7/10
70000/70000 [==============================] - 34s 487us/sample - loss: 0.1428 - categorical_accuracy: 0.9567 - val_loss: 0.1259 - val_categorical_accuracy: 0.9607
Epoch 8/10
70000/70000 [==============================] - 35s 493us/sample - loss: 0.1412 - categorical_accuracy: 0.9574 - val_loss: 0.1314 - val_categorical_accuracy: 0.9636
Epoch 9/10
70000/70000 [==============================] - 34s 489us/sample - loss: 0.1409 - categorical_accuracy: 0.9572 - val_loss: 0.1

70000/70000 [==============================] - 15s 217us/sample - loss: 0.1339 - categorical_accuracy: 0.9597 - val_loss: 0.2771 - val_categorical_accuracy: 0.9632
Epoch 7/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1329 - categorical_accuracy: 0.9601 - val_loss: 0.2064 - val_categorical_accuracy: 0.9562
Epoch 8/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1318 - categorical_accuracy: 0.9600 - val_loss: 0.2641 - val_categorical_accuracy: 0.9645
Epoch 9/10
70000/70000 [==============================] - 15s 219us/sample - loss: 0.1304 - categorical_accuracy: 0.9616 - val_loss: 0.2298 - val_categorical_accuracy: 0.9645
Epoch 10/10
70000/70000 [==============================] - 15s 218us/sample - loss: 0.1315 - categorical_accuracy: 0.9605 - val_loss: 0.4252 - val_categorical_accuracy: 0.9619
Validation CEE:  0.59445
Time passed:  163.2433705329895

Deep Size:  12 , Num of Units:  4 , Learning rate:  0.0064 , Batch Size:  16


70000/70000 [==============================] - 17s 248us/sample - loss: 0.9553 - categorical_accuracy: 0.5948 - val_loss: 0.9561 - val_categorical_accuracy: 0.5936
Validation CEE:  0.59445
Time passed:  119.12306523323059
9  	4     

Deep Size:  16 , Num of Units:  4 , Learning rate:  0.0024000000000000002 , Batch Size:  16
Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 46s 663us/sample - loss: 0.9588 - categorical_accuracy: 0.5948 - val_loss: 0.9563 - val_categorical_accuracy: 0.5936
Epoch 2/10
70000/70000 [==============================] - 41s 591us/sample - loss: 0.9554 - categorical_accuracy: 0.5948 - val_loss: 0.9565 - val_categorical_accuracy: 0.5936
Epoch 3/10
70000/70000 [==============================] - 42s 602us/sample - loss: 0.9553 - categorical_accuracy: 0.5948 - val_loss: 0.9564 - val_categorical_accuracy: 0.5936
Epoch 4/10
70000/70000 [==============================] - 42s 604us/sample - loss: 0.9554 - categor

### Guardar datos

In [72]:
filename = "historial_sdss.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate",
                                    "Batch size", "Epochs", "Error", "Tiempo de ejecución"])

df.sort_values(by=["Error", "Tiempo de ejecución"], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='a') # a=append, w=overwrite

In [73]:
df

,Deep size,Num units,Learning rate,Batch size,Epochs,Error,Tiempo de ejecución
0,12,4,0.0008,32,10,0.16523,212.772860
1,12,4,0.0008,32,10,0.21272,217.912251
2,16,16,0.0008,16,10,0.21326,432.514113
3,12,12,0.0040,16,10,0.21593,407.807038
4,12,8,0.0056,16,10,0.21636,395.011061
5,4,4,0.0008,16,10,0.21682,310.942073
6,4,4,0.0008,16,10,0.21770,309.129902
7,8,12,0.0016,32,10,0.22795,152.763332
8,4,4,0.0032,32,10,0.24065,165.604143
9,8,16,0.0064,16,10,0.24458,351.618607


In [78]:
np.sum(df[["Tiempo de ejecución"]])/60/60

Tiempo de ejecución    3.964932
dtype: float64

### Cargar datos

In [ ]:
pathname = os.path.join('.\\', filename)
while True:
    try: 
        df 
        break
    except:
        df = pd.read_csv(pathname, delimiter = "\t")
        break

In [ ]:
df

### Mejores individuos

In [ ]:
# Selecciona los mejores k individuos - (para k=5)
k = 3
best_genes = df.iloc[:k,]

best_deep_size = best_genes.iloc[:,0]
best_num_units = best_genes.iloc[:,1]
best_learning_rate = best_genes.iloc[:,2]
best_batch_size = best_genes.iloc[:,3]
best_epochs = best_genes.iloc[:,4]

best_genes

In [ ]:
best_deep_size

In [61]:
# Train the model using best configuration on complete training set 
#and make predictions on the test set

X,Y = prepare_dataset(train_data)
X_train, X_test, y_train, y_test = split(X, Y, test_size = 0.20, random_state = 0)

models = []
historial = []
y_pred = []

for k in range(len(best_deep_size)):
    print('\nDeep Size: ', best_deep_size[k], ', Num of Units: ', best_num_units[k], ', Learning rate: ', best_learning_rate[k])
    print('Batch Size: ', best_batch_size[k], ", Num of Epochs: ", best_epochs[k])
    
    models.append(keras.Sequential())
    models[-1].add(keras.Input(shape=(int(X_train.shape[1]),)))
    models[-1].add(layers.Dense(best_num_units[k], input_shape=(int(X_train.shape[1]),)))
#     x = LSTM(num_units, input_shape=(window_size,1))(inputs)

    for i in range(best_deep_size[k]):        
        models[-1].add(layers.Dense(best_num_units[k], activation='relu'))
    models[-1].add(layers.Dense(1, activation='linear'))

    optimizer = keras.optimizers.Adam(learning_rate=best_learning_rate[k], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    models[-1].compile(optimizer=optimizer, loss=metrics[1][m])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                       min_delta=0,
                                       patience=50,
                                       restore_best_weights=True)]
#     history = models[-1].fit(X_train, y_train, validation_data=(X_test, y_test), 
#                         epochs=best_epochs, callbacks=callbacks, batch_size=best_batch_size, shuffle=True)
    historial.append(models[-1].fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=best_epochs[k], batch_size=best_batch_size[k], shuffle=True))
    y_pred.append(models[-1].predict(X))

    error = metrics[0][m](Y, y_pred[-1])
    print('Test RMSE(RMSLE): ', error)

NameError: name 'train_data' is not defined

In [ ]:
fig, axs = plt.subplots(1, len(models), figsize=(14,5), sharey='row')
fig.suptitle('Best models')
titles = []
for k in range(len(models)):
    titles.append(f"Model loss {k+1} (log)")
    axs[k].plot(historial[k].history['loss'])
    axs[k].plot(historial[k].history['val_loss'])
#     print(titles[-1])
    axs[k].set_title(titles[-1])
    axs[k].set_yscale("log")
    axs[k].legend(['train', 'test'], loc='upper left')
    axs[k].grid()

for ax in axs.flat:
    ax.set(xlabel='Epoch', ylabel='Loss')
plt.show()

### Gráficas de comparación

In [ ]:
Y_pred1 = models[0].predict(X)
Y_pred2 = models[1].predict(X)
Y_pred3 = models[2].predict(X)

In [ ]:
fig, axs = plt.subplots(len(models), figsize=(14,9), sharex="col")
fig.suptitle('Best models')

axs[0].scatter(X[:,0], Y, s=1e1, alpha=0.8, c='red', label="Data")
axs[0].set_yscale("log"); axs[0].grid(); axs[0].legend()

axs[1].scatter(X[:,0], Y_pred1, s=1e1, alpha=0.8, c='blue', label="Model pred 1")
axs[1].set_yscale("log"); axs[1].grid(); axs[1].legend()

axs[2].scatter(X[:,0], Y_pred3, s=1e1, alpha=0.8, c='purple', label="Model pred 3")
axs[2].set_yscale("log"); axs[2].grid(); axs[2].legend()

# axs[3].scatter(X[:,0], Y_pred3, s=1e1, alpha=0.8, c='green', label="Model pred 3")
# axs[3].set_yscale("log"); axs[3].grid(); axs[3].legend()

for ax in axs.flat:
    ax.set(xlabel=r'$k$', ylabel='$A(k,r,x)$')
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(X[:,0], Y, s=5e1, alpha=0.8, c='red', label="Data")
# plt.scatter(X[:,0], Y_pred1, s=1e1, alpha=0.4, c='blue', label="Model pred 1")
plt.scatter(X[:,0], Y_pred3, s=1e1, alpha=0.4, c='green', label="Model pred 3")
plt.xlabel('$k$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(len(models), figsize=(14,9), sharex="col")
fig.suptitle('Data and model predictions')

axs[0].scatter(X[:,1], Y, s=1e1, alpha=0.8, c='red', label="Data")
axs[0].set_yscale("log"); axs[0].grid(); axs[0].legend()

axs[1].scatter(X[:,1], Y_pred1, s=1e1, alpha=0.8, c='blue', label="Model pred 1")
axs[1].set_yscale("log"); axs[1].grid(); axs[1].legend()

axs[2].scatter(X[:,1], Y_pred3, s=1e1, alpha=0.8, c='purple', label="Model pred 3")
axs[2].set_yscale("log"); axs[2].grid(); axs[2].legend()

for ax in axs.flat:
    ax.set(xlabel=r'$r$', ylabel='$A(k,r,x)$')
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
plt.scatter(X[:,1], Y, s=5e0, alpha=0.7, c='red', label="Data")
plt.scatter(X[:,1], Y_pred3, s=5e0, alpha=0.7, c='green', label="Model pred 3")
# plt.scatter(X[:,1], Y_pred2, s=5e0, alpha=0.7, c='blue', label="Model pred 2")
plt.xlabel('$r$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend(); plt.grid();
plt.show()

In [ ]:
fig, axs = plt.subplots(1,len(models), figsize=(14,5), sharey="row")
fig.suptitle('Data and model predictions')

axs[0].scatter(X[:,2], Y, s=1e0, alpha=0.8, c='red', label="Data")
axs[0].set_yscale("log"); axs[0].grid(); axs[0].legend()

axs[1].scatter(X[:,2], Y_pred1, s=1e0, alpha=0.8, c='blue', label="Model pred 1")
axs[1].set_yscale("log"); axs[1].grid(); axs[1].legend()

axs[2].scatter(X[:,2], Y_pred3, s=1e0, alpha=0.8, c='purple', label="Model pred 3")
axs[2].set_yscale("log"); axs[2].grid(); axs[2].legend()

for ax in axs.flat:
    ax.set(xlabel=r'$x$', ylabel='$A(k,r,x)$')
plt.show()

### Comparación 3D

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Data")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y_pred1, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Model prediction 1")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y_pred3, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Model prediction 3")
plt.show()

### Resultado de hiperparámetros

In [ ]:
import seaborn as sns

In [ ]:
df

In [ ]:
# Guarda las mejores N soluciones - (1, para k=1)
best_individuals2 = tools.selBest(population,k = 20)
deep_size2 = []
num_units2 = []
learning_rate2 = []
batch_size2 = []
epochs2 = []
t = 0

for bi in best_individuals2:
    deep_size_bits2 = BitArray(bi[0:2])
    num_units_bits2 = BitArray(bi[2:4])
    learning_rate_bits2 = BitArray(bi[4:7])
    batch_size_bits2 = BitArray(bi[7:9])
    epochs_bits2 = BitArray(bi[9:])
    t += 1 
    
    deep_size2.append( 4*(deep_size_bits2.uint + 1))
    num_units2.append( 4*(num_units_bits2.uint + 1))
    learning_rate2.append( 8*(learning_rate_bits2.uint+1)*10**(-4))
    batch_size2.append( 2**(batch_size_bits2.uint +1))
    epochs2.append( 50*(epochs_bits2.uint +1))
#     print('k=',t,'\nDeep Size: ', best_deep_size2[-1], ', Num of Units: ', best_num_units2[-1], ', Learning rate: ', best_learning_rate2[-1])
#     print('Batch Size: ', best_batch_size2[-1], ", Num of Epochs: ", best_epochs2[-1])

In [ ]:

best_genes.iloc[0]

In [ ]:
# fig, axs = plt.subplots(1,2, figsize=(15,5))

# fig.suptitle('Última generación de hipermarámetros obtenidos')

# # axs[0].plot(df.iloc[:,0], df.iloc[:,1], alpha=0.5, c='blue', label="Model pred 1")
# axs[0].scatter(df.iloc[:,0], df.iloc[:,1], s=20, alpha=0.3, c='red', label="Model pred 1")
# axs[0].scatter(best_genes.iloc[:,0], best_genes.iloc[:,1], s=80, alpha=1, c='red', label="Mejores individuos")
# axs[0].grid(); axs[0].legend()
# # axs[0].legend(loc='upper right'); 
# # axs[0].set_xlim([2, 18]); axs[0].set_ylim([1, 65])
# axs[0].set(xlabel=r'Deep size', ylabel='Num units')

# # axs[1].plot(df.iloc[:,2], df.iloc[:,3], alpha=0.5, c='blue', label="Model pred 1")
# axs[1].scatter(df.iloc[:,2], df.iloc[:,3], s=20, alpha=0.3, c='red', label="Model pred 1")
# axs[1].scatter(best_genes.iloc[:,2], best_genes.iloc[:,3], s=80, alpha=1, c='red', label="Mejores individuos")
# axs[1].set_yscale("log"); 
# axs[1].grid(); axs[1].legend(loc='upper right')
# # axs[1].set_xlim([1e-4, 31e-4]); axs[1].set_ylim([2**1, 2**4])
# axs[1].set(xlabel=r'Learning rate', ylabel='Batch size')

In [ ]:
g1= sns.jointplot(data=df, x="Deep size", y="Num units", kind="kde")
g1.ax_joint.scatter(best_genes.iloc[:,0], best_genes.iloc[:,1], color = 'red', label="a")

plt.show()

g2= sns.jointplot(x=df.iloc[:,2], y=df.iloc[:,3], kind='kde')
g2.ax_joint.scatter(best_genes.iloc[:,2], best_genes.iloc[:,3], color = 'red', label="a")

plt.show()